In [1]:
%cd /content/
%rm -rf LLaMA-Factory
!git clone  https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install -e .[torch,bitsandbytes]

/content
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 15748, done.
remote: Counting objects: 100% (6778/6778), done.
remote: Compressing objects: 100% (478/478), done.
remote: Total 15748 (delta 6425), reused 6327 (delta 6300), pack-reused 8970
Receiving objects: 100% (15748/15748), 221.88 MiB | 13.64 MiB/s, done.
Resolving deltas: 100% (11616/11616), done.
/content/LLaMA-Factory
assets/       docker/      LICENSE      pyproject.toml  requirements.txt  src/
CITATION.cff  evaluation/  Makefile     README.md       scripts/          tests/
data/         examples/    MANIFEST.in  README_zh.md    setup.py
Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Check GPU enviroment
import torch
try:
  assert torch.cuda.is_available() is True
except:
  print("GPU not available")

In [4]:
import json

# تغییر مسیر به دایرکتوری مورد نظر
%cd /content/LLaMA-Factory/

# تعریف نام و نویسنده
NAME = "Llama-3"
AUTHOR = "LLaMA-Factory"

with open("data/identity.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

for sample in dataset:
    sample["output"] = sample["output"].replace(" {{" + "name" + "}}", NAME).replace("{{"+ "author"+ "}}", AUTHOR)

with open("data/identity.json", "w", encoding="utf-8") as f:
    json.dump(dataset, f, indent=2, ensure_ascii=False)

/content/LLaMA-Factory


In [8]:
import json

args = dict(
    stage="sft",  # do supervised fine-tuning
    do_train=True,
    model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit",  # use bnb 4bit quantized llama 3b
    dataset="identity,alpaca_en_demo",  # use alpaca and identity datasets
    template="llama3",  # use llama prompt template
    finetuning_type="lora",  # use LoRA adapters to save memory
    lora_target="all",  # attach LoRA adapters to all linear layers
    output_dir="llama3_lora",  # the path to save LoRA adapters
    per_device_train_batch_size=2,  # the batch size
    gradient_accumulation_steps=4,  # the gradient accumulation steps
    lr_scheduler_type="cosine",  # use cosine learning rate scheduler
    logging_steps=10,  # log every 10 steps
    warmup_ratio=0.1,  # the warmup scheduler
    save_steps=1000,  # save checkpoint every 1000 steps
    learning_rate=5e-5,  # the learning rate
    num_train_epochs=3.0,  # the epochs of training
    max_samples=500,  # use 500 samples in each dataset
    max_grad_norm=1.0,  # the gradient norm to 1.0
    quantization_bit=4,  # use 4 bit QLoRA
    loraplus_lr_ratio=16.0,  # llm lora algorithm with lambda=16.0
    fp16=True,  # floating mixed precision training
)

json.dump(args, open("train_llama3.json", "w", encoding="utf-8"), indent=2)

%cd /content/LLAMA-Factory/

!llamafactory-cli train train_llama3.json

[Errno 2] No such file or directory: '/content/LLAMA-Factory/'
/content/LLaMA-Factory
2024-07-18 17:44:20.995051: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-18 17:44:20.995104: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-18 17:44:20.996466: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-18 17:44:21.003595: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorF

In [10]:
!pip install git+https://github.com/hiyouga/LLaMA-Factory.git # Install LLaMa-Factory using the correct repo link

  Cloning https://github.com/hiyouga/LLaMA-Factory.git to /tmp/pip-req-build-azawewq9
  Running command git clone --filter=blob:none --quiet https://github.com/hiyouga/LLaMA-Factory.git /tmp/pip-req-build-azawewq9
  Resolved https://github.com/hiyouga/LLaMA-Factory.git to commit 88c7fc159999511e6e19fff3d37147a6a7064335
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llamafactory: filename=llamafactory-0.8.4.dev0-py3-none-any.whl size=231858 sha256=055ab66360f8c75591bf2cbd0601834b99f93093b7b24d34b2ed9c5637adf0e5
  Stored in directory: /tmp/pip-ephem-wheel-cache-29at0mk1/wheels/6f/e2/1c/0951237e1114b7824788ea6e772820897ee0f6506afec796bb
Successfully built llamafactory
  Attempting uninstall: llamafactory
    Found existing installation: llamafactory 0.8.3
    Uninstalling llamafactory-0.8.3:
      Successfully uninstalled llamafactory-0.8.3


In [ ]:
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc

%cd /content/LLAMA-Factory/

args = dict(
    model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit",  # use bnb 4bit quantized llama-3b-8bit Instruct model
    adapter_name_or_path="llama3_lora",  # load with saved LoRA adapters
    template="llama3",  # same as the one in training
    finetuning_type="lora",  # same as the one in training
    quantization_bit=4,  # 4-bit quantized model
)

chat_model = ChatModel(args)

messages = []
print("Welcome to the CLI application, use `clear` to remove the history, use `exit` to exit the application.")
while True:
    query = input("\nUser: ")
    if query.strip() == "exit":
        break
    if query.strip() == "clear":
        messages = []
        torch_gc()
        print("History has been removed.")
        continue

    messages.append({"role": "user", "content": query})
    print("Assistant: ", end="", flush=True)

    response = ""
    for new_text in chat_model.stream_chat(messages):
        print(new_text, end='', flush=True)
        response += new_text
    print()
    messages.append({"role": "assistant", "content": response})

torch_gc()

[Errno 2] No such file or directory: '/content/LLAMA-Factory/'
/content/LLaMA-Factory


[INFO|tokenization_utils_base.py:2161] 2024-07-18 18:31:22,193 >> loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/e11416454f0acc5f2fc66f75c209b7786eaeb956/tokenizer.json
[INFO|tokenization_utils_base.py:2161] 2024-07-18 18:31:22,194 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2161] 2024-07-18 18:31:22,197 >> loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/e11416454f0acc5f2fc66f75c209b7786eaeb956/special_tokens_map.json
[INFO|tokenization_utils_base.py:2161] 2024-07-18 18:31:22,199 >> loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/e11416454f0acc5f2fc66f75c209b7786eaeb956/tokenizer_config.json
[WARNING|logging.py:313] 2024-07-18 18:31:22,572 >> Special tokens have been added in the vocabulary, mak

07/18/2024 18:31:22 - INFO - llamafactory.data.template - Replace eos token: <|eot_id|>


INFO:llamafactory.data.template:Replace eos token: <|eot_id|>
[INFO|configuration_utils.py:733] 2024-07-18 18:31:22,814 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/e11416454f0acc5f2fc66f75c209b7786eaeb956/config.json
[INFO|configuration_utils.py:800] 2024-07-18 18:31:22,817 >> Model config LlamaConfig {
  "_name_or_path": "unsloth/llama-3-8b-Instruct-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bn

07/18/2024 18:31:22 - WARNING - llamafactory.model.model_utils.quantization - `quantization_bit` will not affect on the PTQ-quantized models.


...

07/18/2024 18:31:36 - INFO - llamafactory.model.adapter - Loaded adapter(s): llama3_lora


INFO:llamafactory.model.adapter:Loaded adapter(s): llama3_lora


07/18/2024 18:31:36 - INFO - llamafactory.model.loader - all params: 8,051,232,768


INFO:llamafactory.model.loader:all params: 8,051,232,768


Welcome to the CLI application, use `clear` to remove the history, use `exit` to exit the application.

User: "Who are you?"
Assistant: I amLlama-3, an AI assistant developed by LLaMA-Factory.

User: "Can you introduce yourself?",
Assistant: I amLlama-3, an AI assistant developed by LLaMA-Factory.
